# DistLRE usage
This notebook presents a method of using the DistLRE package within the UNH AI group in order to run commands on the remote AI machines.

In [1]:
import getpass
from slack_notification import start_experiment_notification, end_experiment_notification
from distlre.distlre import DistLRE, Task, RemoteHost

## Set up and logging into the machines
By default the default user you are logged in as will be used, we recommend executing the script from a machine already logged into the AI subnet. Collect the user's password and log on to each of the machines. Don't forget to notify the `#experiments` channel on Slack!

In [2]:
HOSTS = ['ai' + str(i) + '.cs.unh.edu' for i in [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]]
password = getpass.getpass("Password to connect to [ai.cs.unh.edu]")
remote_hosts = [RemoteHost(host, port=22, password=password) for host in HOSTS]
executor = DistLRE(remote_hosts=remote_hosts)

Password to connect to [ai.cs.unh.edu] ············


## Securely running the commands
In order to prevent users from clobbering each other's work we all need to adhere to a standard. For simplicity, if there is a `distlre.lock` file located at the shared directory `/home/aifs2/group/exp` we should not execute the experiments. 

In [3]:
import os.path
from datetime import datetime
file_lock_path = '/home/aifs2/group/exp/distlre.lock'

We can define a function which will always check the `.lock` file, prior to executing the commmands.

In [4]:
def execute_tasks():
    if not os.path.isfile(file_lock_path):
        lock_file = open(file_lock_path, 'w')
        lock_file.write(getpass.getuser() + '@' + str(datetime.now()))
        lock_file.close()
        executor.execute_tasks()
        executor.wait()
        os.remove(file_lock_path)
    else:
        lock_file = open(file_lock_path, 'r')
        contents = lock_file.read().strip().split('@')
        user, time = (contents[0], contents[1])
        raise Exception(f"{user} started experiments @ {time}, check #experiments on Slack for updates")

## Checking the idle state
We should check that the machines are in a mostly idle state, lets use 98%.

In [5]:
vmstat_command = "vmstat 2 2"
commands = [(vmstat_command, machine) for machine in HOSTS]
tasks = [Task(command=command, meta=f"VMcheck@{host}", time_limit=10, memory_limit=10) for command, host in commands]
futures = []
for task in tasks:
    future = executor.submit(task)
    futures.append(future)
execute_tasks()
for future in futures:
    result = future.result()
    output = result.output.split('\n')
    last_line = output[-2].split()
    idle = last_line[-3]
    assert int(idle) >= 98
    print(result.meta, "is idle!")

VMcheck@ai1.cs.unh.edu is idle!
VMcheck@ai2.cs.unh.edu is idle!
VMcheck@ai3.cs.unh.edu is idle!
VMcheck@ai4.cs.unh.edu is idle!
VMcheck@ai5.cs.unh.edu is idle!
VMcheck@ai6.cs.unh.edu is idle!
VMcheck@ai8.cs.unh.edu is idle!
VMcheck@ai9.cs.unh.edu is idle!
VMcheck@ai10.cs.unh.edu is idle!
VMcheck@ai11.cs.unh.edu is idle!
VMcheck@ai12.cs.unh.edu is idle!
VMcheck@ai13.cs.unh.edu is idle!
VMcheck@ai14.cs.unh.edu is idle!
VMcheck@ai15.cs.unh.edu is idle!


## Adding commands to the run queue
Now that we know the machines are idling, we can next create each of our commands to run. For simplicity we are simply performing an `echo`, these commands will hold the line which you want to run on the machine and place it into the queue.

In [6]:
remote_hosts = [RemoteHost(host, port=22, password=password) for host in HOSTS]
executor = DistLRE(remote_hosts=remote_hosts)
command_to_run = "echo My Super Awesome Experiment "
commands = [f"{command_to_run}{experiment_number}!!!" for experiment_number in range(1, 24)]
start_experiment_notification(experiment_count=len(commands))

In [7]:
commands

['echo My Super Awesome Experiment 1!!!',
 'echo My Super Awesome Experiment 2!!!',
 'echo My Super Awesome Experiment 3!!!',
 'echo My Super Awesome Experiment 4!!!',
 'echo My Super Awesome Experiment 5!!!',
 'echo My Super Awesome Experiment 6!!!',
 'echo My Super Awesome Experiment 7!!!',
 'echo My Super Awesome Experiment 8!!!',
 'echo My Super Awesome Experiment 9!!!',
 'echo My Super Awesome Experiment 10!!!',
 'echo My Super Awesome Experiment 11!!!',
 'echo My Super Awesome Experiment 12!!!',
 'echo My Super Awesome Experiment 13!!!',
 'echo My Super Awesome Experiment 14!!!',
 'echo My Super Awesome Experiment 15!!!',
 'echo My Super Awesome Experiment 16!!!',
 'echo My Super Awesome Experiment 17!!!',
 'echo My Super Awesome Experiment 18!!!',
 'echo My Super Awesome Experiment 19!!!',
 'echo My Super Awesome Experiment 20!!!',
 'echo My Super Awesome Experiment 21!!!',
 'echo My Super Awesome Experiment 22!!!',
 'echo My Super Awesome Experiment 23!!!']

In [8]:
tasks = [Task(command=command, meta='META', time_limit=10, memory_limit=10) for command in commands]

In [9]:
futures = []
for task in tasks:
    future = executor.submit(task)
    futures.append(future)
execute_tasks()

## View the results
The futures contain the initial `Task` created when adding items to the run queue. As a result, we can view its various stats including:

`meta, command, input, output, error, time_limit, memory_limit`

The only info from the `echo` command we are interested in here is `output`, so lets diplay it.

In [10]:
for future in futures:
    result = future.result()
    print(result.output)
end_experiment_notification()

My Super Awesome Experiment 1!!!

My Super Awesome Experiment 2!!!

My Super Awesome Experiment 3!!!

My Super Awesome Experiment 4!!!

My Super Awesome Experiment 5!!!

My Super Awesome Experiment 6!!!

My Super Awesome Experiment 7!!!

My Super Awesome Experiment 8!!!

My Super Awesome Experiment 9!!!

My Super Awesome Experiment 10!!!

My Super Awesome Experiment 11!!!

My Super Awesome Experiment 12!!!

My Super Awesome Experiment 13!!!

My Super Awesome Experiment 14!!!

My Super Awesome Experiment 15!!!

My Super Awesome Experiment 16!!!

My Super Awesome Experiment 17!!!

My Super Awesome Experiment 18!!!

My Super Awesome Experiment 19!!!

My Super Awesome Experiment 20!!!

My Super Awesome Experiment 21!!!

My Super Awesome Experiment 22!!!

My Super Awesome Experiment 23!!!

